In [1]:
import pandas as pd

# 데이터 불러오기
match_data = pd.read_csv('/Users/lhe339/Documents/GitHub/nextorial/data/데이터 분석가_과제_Type A/match_data.csv')

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.10 from "/Users/lhe339/Documents/GitHub/nextorial/.conda/bin/python"
  * The NumPy version is: "1.23.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'


In [25]:
def calculate_team_features(data):
    return data.groupby(['matchid', 'teamid']).agg({
        'mmr': 'mean',
        'winstreak': 'mean',
        'losestreak': 'mean',
        'recentwinprob': 'mean'
    }).reset_index()

# match_data에 대한 전처리
match_team_features = calculate_team_features(match_data)

# 원래 match_data와 merge하여 각 유저 옆에 팀의 평균 특성값 추가하기
match_data = match_data.merge(match_team_features, on=['matchid', 'teamid'], suffixes=('', '_team_avg'))


,createdatekst,matchid,accountid,teamid,mmr,guildid,winstreak,losestreak,recentwinprob,accumatches,tier,itemid,matchresult,matchscore,isDrop,isEscape,mmr_team_avg,winstreak_team_avg,losestreak_team_avg,recentwinprob_team_avg
0,2022-07-04 00:27:58,m00160017353d8e81,2097311063,1,1812,G69102,1,0,0.428571,7,gold,1364,0,25,1,0,1780.0,1.00,0.50,0.315476
1,2022-07-04 00:27:58,m00160017353d8e81,1560653129,1,1868,NaN,1,0,0.333333,9,gold,1178,0,35,0,0,1780.0,1.00,0.50,0.315476
2,2022-07-04 00:27:58,m00160017353d8e81,872694401,1,1809,G5900,0,2,0.200000,162,platinum,1385,0,39,0,0,1780.0,1.00,0.50,0.315476
3,2022-07-04 00:27:58,m00160017353d8e81,84538717,1,1631,G424,2,0,0.300000,11,bronze,1338,0,0,1,0,1780.0,1.00,0.50,0.315476
4,2022-07-04 00:27:58,m00160017353d8e81,142829,2,1973,NaN,0,3,0.500000,412,platinum,1277,1,41,0,0,1890.5,1.25,1.75,0.425000


In [26]:
def guild_avg(df):
    # 'guildid'에서 NaN 값을 처리합니다. 이 예에서는 'NoGuild'로 대체합니다.
    df['guildid'].fillna('NoGuild', inplace=True)

    # 티어를 숫자로 변환합니다.
    tier_mapping = {'unranked': 0, 'bronze': 1, 'silver': 2, 'gold': 3, 'platinum': 4, 'diamond': 5, 'master': 6}
    df['tier_numeric'] = df['tier'].map(tier_mapping)

    # 길드별로 데이터를 그룹화하고 그룹별로 평균을 계산합니다.
    guild_avg = df.groupby('guildid')[['mmr', 'winstreak', 'recentwinprob', 'accumatches', 'tier_numeric']].mean()

    # 새로운 컬럼 이름을 생성합니다.
    new_columns = {col: f'{col}_guild_avg' for col in guild_avg.columns}
    guild_avg.rename(columns=new_columns, inplace=True)

    # 원본 데이터셋에 계산된 길드 성능 지표를 병합합니다.
    df = pd.merge(df, guild_avg, on='guildid', how='left')
    
    return df


match_data = guild_avg(match_data)

In [27]:
match_data

,createdatekst,matchid,accountid,teamid,mmr,guildid,winstreak,losestreak,recentwinprob,accumatches,...,matchresult,matchscore,isDrop,isEscape,tier_numeric,mmr_guild_avg,winstreak_guild_avg,recentwinprob_guild_avg,accumatches_guild_avg,tier_numeric_guild_avg
0,2022-07-04 00:27:58,m00160017353d8e81,2097311063,1,1812,G69102,1,0,0.428571,7,...,0,25,1,0,3,1887.000000,0.333333,0.459656,4.444444,3.000000
1,2022-07-04 00:27:58,m00160017353d8e81,1560653129,1,1868,NoGuild,1,0,0.333333,9,...,0,35,0,0,3,1994.394511,0.991028,0.490311,393.967106,3.740939
2,2022-07-04 00:27:58,m00160017353d8e81,872694401,1,1809,G5900,0,2,0.200000,162,...,0,39,0,0,4,1918.224719,0.617978,0.416854,149.646067,4.000000
3,2022-07-04 00:27:58,m00160017353d8e81,84538717,1,1631,G424,2,0,0.300000,11,...,0,0,1,0,1,1680.200000,0.500000,0.311690,79.940000,1.860000
4,2022-07-04 00:27:58,m00160017353d8e81,142829,2,1973,NoGuild,0,3,0.500000,412,...,1,41,0,0,4,1994.394511,0.991028,0.490311,393.967106,3.740939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410379,2022-06-27 01:47:35,m03c40017117a01f2,621010036,1,1990,G45703,0,5,0.500000,191,...,1,44,0,0,4,1920.273292,1.018634,0.478261,208.782609,4.000000
410380,2022-06-27 01:47:35,m03c40017117a01f2,1711756742,2,2050,G185081,0,1,0.500000,1201,...,0,45,0,0,4,2017.380952,0.571429,0.404762,1204.000000,4.000000
410381,2022-06-27 01:47:35,m03c40017117a01f2,1460484825,2,2055,NoGuild,0,1,0.500000,139,...,0,49,0,0,1,1994.394511,0.991028,0.490311,393.967106,3.740939
410382,2022-06-27 01:47:35,m03c40017117a01f2,218329344,2,1966,G28051,0,2,0.300000,389,...,0,48,0,0,4,2012.675000,0.850000,0.480000,400.500000,4.000000
